# Worksheet 08

Name: Akuraju Mounika Chowdary

UID: U70197123 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [12]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.163215444850255, 4.822197435284785, 6.024418092659602, 4.618831909147721, 4.6944751656604975, 6.363518825978492, 4.712916536853552, 4.845140022748724, 6.15655298500748, 5.926149007175853]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [13]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[10.60294109711851, 8.157059232458467, 5.671913998633125, 8.940878850860294, 8.754905260348725, 8.349616519859286, 8.04214457838085, 7.047974282169902, 8.51105771647338, 7.6484871758401765]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [14]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.926149007175853, 7.6484871758401765, 6.15655298500748, 4.845140022748724, 8.51105771647338, 7.047974282169902, 8.04214457838085, 4.712916536853552, 8.349616519859286, 8.754905260348725]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [15]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_c = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.6484871758401765, 8.51105771647338, 7.047974282169902, 8.04214457838085, 8.349616519859286, 8.754905260348725]
[5.926149007175853, 6.15655298500748, 4.845140022748724, 4.712916536853552]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.059030922178719,  mean_2 = 5.410189637946402
var_1 = 0.3273460254056146,  var_2 = 0.4071857904186381


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [16]:
from scipy.stats import norm

prob_c0_x = [] # P(S_0 | X_i)
prob_c1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_c[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.926149007175853
probability of observing that point if it came from cluster 0 =  7.364032408172932e-10
probability of observing that point if it came from cluster 1 =  0.4389945039344781
point =  7.6484871758401765
probability of observing that point if it came from cluster 0 =  0.5550719064278085
probability of observing that point if it came from cluster 1 =  2.6890164112268785e-07
point =  6.15655298500748
probability of observing that point if it came from cluster 0 =  5.6396533244843415e-08
probability of observing that point if it came from cluster 1 =  0.18261770054457965
point =  4.845140022748724
probability of observing that point if it came from cluster 0 =  1.4266018687578742e-21
probability of observing that point if it came from cluster 1 =  0.37407413093067654
point =  8.51105771647338
probability of observing that point if it came from cluster 0 =  0.4697188481205414
probability of observing that point if it came from cluster 1 =  2.499942350426185e-13
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [17]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]

mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]

var = [sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * ((x[1] - mean[1]) ** 2) for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5980977314637358,  P(S_2) = 0.4019022685362642
mean_1 = 8.06224651741924,  mean_2 = 5.417941689770441
var_1 = 0.32512581637312993,  var_2 = 0.4178947966084489


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [18]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        # print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        # prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


5.926149007175853
Probability of coming from C_1 = 1.6962530854464317e-09
Probability of coming from C_2 = 0.999999998303747

7.6484871758401765
Probability of coming from C_1 = 0.9999992352015531
Probability of coming from C_2 = 7.647984469828767e-07

6.15655298500748
Probability of coming from C_1 = 3.160046524365134e-07
Probability of coming from C_2 = 0.9999996839953476

4.845140022748724
Probability of coming from C_1 = 2.6835322357070447e-21
Probability of coming from C_2 = 1.0

8.51105771647338
Probability of coming from C_1 = 0.9999999999982789
Probability of coming from C_2 = 1.7210634922291866e-12

7.047974282169902
Probability of coming from C_1 = 0.9673837326993834
Probability of coming from C_2 = 0.032616267300616626

8.04214457838085
Probability of coming from C_1 = 0.9999999985665762
Probability of coming from C_2 = 1.4334237302211e-09

4.712916536853552
Probability of coming from C_1 = 7.165167626827546e-23
Probability of coming from C_2 = 1.0

8.349616519859286
Probabi

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [19]:
probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])

    if p[1] > p[2]:
        print("belongs to cluster 1\n")
    elif p[1] < p[2]:
        print("belongs to cluster 2\n")
    else:
        print("Ambiguous\n")
    


5.926149007175853
belongs to cluster 2

7.6484871758401765
belongs to cluster 1

6.15655298500748
belongs to cluster 2

4.845140022748724
belongs to cluster 2

8.51105771647338
belongs to cluster 1

7.047974282169902
belongs to cluster 1

8.04214457838085
belongs to cluster 1

4.712916536853552
belongs to cluster 2

8.349616519859286
belongs to cluster 1

8.754905260348725
belongs to cluster 1

